SQLite DB:  Storing and Retrieving Vectors

In [ ]:
# !pip install sqlite3
import sqlite3
import numpy as np

In [ ]:
# create a connection to the sqlite DB
conn = sqlite3.connect('vectors-db.db')

# create a cursor object to execute sql commands
cursor = conn.cursor()

In [ ]:
# create a table for the vector data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS vectors (
        id INTEGER PRIMARY KEY,
        vector BLOB NOT NULL
    )
''')

In [ ]:
# generate some sample vectors
vect1 = np.array([1.2, 2.3, 3.1,0.8])
vect2 = np.array([4.7, 1.5, 3.9,2.6])

In [ ]:
vect1.tobytes()   # numpy array to bystream

b'333333\xf3?ffffff\x02@\xcd\xcc\xcc\xcc\xcc\xcc\x08@\x9a\x99\x99\x99\x99\x99\xe9?'

In [ ]:
vect2.tobytes()

b'\xcd\xcc\xcc\xcc\xcc\xcc\x12@\x00\x00\x00\x00\x00\x00\xf8?333333\x0f@\xcd\xcc\xcc\xcc\xcc\xcc\x04@'

In [ ]:
# insert vector data into table
cursor.execute("INSERT INTO vectors(vector) values(?)",
               (sqlite3.Binary(vect1.tobytes()),))
cursor.execute("INSERT INTO vectors(vector) values(?)",
               (sqlite3.Binary(vect2.tobytes()),))

In [ ]:
# Retrieving data
cursor.execute("SELECT * FROM vectors")

In [ ]:
rows = cursor.fetchall()

In [ ]:
rows

[(1,
  b'333333\xf3?ffffff\x02@\xcd\xcc\xcc\xcc\xcc\xcc\x08@\x9a\x99\x99\x99\x99\x99\xe9?'),
 (2,
  b'\xcd\xcc\xcc\xcc\xcc\xcc\x12@\x00\x00\x00\x00\x00\x00\xf8?333333\x0f@\xcd\xcc\xcc\xcc\xcc\xcc\x04@')]

In [ ]:
vector = np.frombuffer(rows[0][1], dtype=np.float64)

In [ ]:
vector

array([1.2, 2.3, 3.1, 0.8])

In [ ]:
for row in rows:
    vector = np.frombuffer(row[1], dtype=np.float64)
    print(vector)

[1.2 2.3 3.1 0.8]
[4.7 1.5 3.9 2.6]


In [ ]:
vectors = []
for row in rows:
    vector = np.frombuffer(row[1], dtype=np.float64)
    vectors.append(vector)

In [ ]:
vectors

[array([1.2, 2.3, 3.1, 0.8]), array([4.7, 1.5, 3.9, 2.6])]

Vector Similarity Search(VSS)

In [ ]:
query_vector = np.array([1.0, 2.0, 3.0, 4.0])

In [ ]:
cursor.execute("""select vector from vectors ORDER BY abs(vector - ?) ASC""",
 (sqlite3.Binary(query_vector.tobytes()),))

In [ ]:
res = cursor.fetchone()  # finding the top one

In [ ]:
np.frombuffer(res[0], dtype=np.float64)  # most similar vector

array([4.7, 1.5, 3.9, 2.6])

In [ ]:
conn.commit()

In [ ]:
conn.close()